In [ ]:
!pip install PersianStemmer
!pip install transformers
!pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade
!pip install hazm
!pip install demoji

In [ ]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0, '/content/drive/My Drive/modules')

In [ ]:
import pandas as pd
import preprocess
import numpy as np
import tensorflow as tf
import demoji
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
np.random.seed(42)
tf.random.set_seed(42)
demoji.download_codes()

In [ ]:
def create_vector_from_data(train_data_df, test_data_df):
    train_sentence_list = [row.sentence for index, row in train_data_df.iterrows()]
    train_labels_list = [row.label for index, row in train_data_df.iterrows()]
    tokenizer = Tokenizer(oov_token='<UNK>')
    tokenizer.fit_on_texts(train_sentence_list)
    train_sequences = tokenizer.texts_to_sequences(train_sentence_list)
    maxlen = max([len(x) for x in train_sequences])
    train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=maxlen)

    test_sentence_list = [row.sentence for index, row in test_data_df.iterrows()]
    test_labels_list = [row.label for index, row in test_data_df.iterrows()]
    test_sequences = tokenizer.texts_to_sequences(test_sentence_list)
    test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=maxlen)
    return train_padded, train_labels_list, test_padded, test_labels_list, len(tokenizer.word_index)+1, maxlen

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/data/initial_dataset.csv')
dataset = preprocess.auto_preprocess(dataframe=dataset, stop_words_type='custom', stemming = True)
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
train_vectors, train_labels_list, test_vectors, test_labels_list, vocab_size, maxlen = create_vector_from_data(train_data_df=train, test_data_df=test)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_vectors, train_labels_list))
train_data = dataset.shuffle(100).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_vectors, test_labels_list))
test_data = test_dataset.batch(64)

In [ ]:
checkpoint_filepath = './'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=maxlen, mask_zero = True))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])
model.summary()
history = model.fit(train_data, epochs=6, validation_data = test_data, class_weight={0:1, 1:5}, callbacks=[model_checkpoint_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 108, 50)           168350    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 197,855
Trainable params: 197,855
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
50/50 [==============================] - 8s 43ms/step - loss: 1.1316 - accuracy: 0.7861 - val_loss: 0.6500 - val_accuracy: 0.7544
Epoch 2/6
50/50 [==============================] - 1s 15ms/step - loss: 0.7851 - accuracy: 0.8450 - val_loss: 0.3739 - val_accuracy: 0.8897
Epoch 3/6
50/50 [==============================] - 1s 16ms/step - loss: 0.3679 - accuracy: 0.9492 - 

In [ ]:
# keras.utils.plot_model(model)

In [ ]:
plot_history(history)

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
# results = model.predict_classes(test_data)
results = (model.predict(test_data) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
f1 = (f1_score(test_labels_list,results ))*100
f1 = round(f1,2)
print('f1 score is: ',f1)

accuracy = (accuracy_score(test_labels_list,results ))*100
accuracy = round(accuracy,2)
print('accuracy score is: ',accuracy)

precision = (precision_score(test_labels_list,results ))*100
precision = round(precision,2)
print('precision score is: ',precision)

recall = (recall_score(test_labels_list,results ))*100
recall = round(recall,2)
print('recall score is: ',recall)


f1 score is:  74.81
accuracy score is:  91.73
precision score is:  80.33
recall score is:  70.0
